Para insertar código .py 

In [ ]:
cd catkin_ws
cd src
catkin_create_pkg movement std_msgs rospy roscpp

Adentro de la carpeta movement , se habrá creado otra llamada src
Allí adentro poner todos los códigos .py

In [ ]:
rosrun movement name_codigo.py

Tópicos : Publisher

In [ ]:
#Comandos
rostopic list#Lista todos los tópicos que se están ejecutando en simulación
rostopic list | grep counter #Filtra el tópico llamaso counter

Info, nos brinda información del tópico: 
Importante:
Nos dice que tipo de mensaje usa Int32, etc
Cuáles son sus publicadores
Cuáles son sus subscriptores
Necesitaremos esta info, cuando queramos hacer nuestro "publisher"

In [ ]:
rostopic info /counter 

Si counter no te aparece, tendrás que correr primero el código de abajo.

echo Imprime toda la información que está siendo publicada en el Tópico

In [ ]:
rostopic echo /counter 
rostopic echo /counter -n1 #Imprime solo la última línea

In [ ]:
rostopic -h #Nos dice todos los comandos relacionados a Tópicos

¿Qué es un publisher?: Es un nodo que publica un "mensaje" en un tópico

Tópico: Es como un pipe que transporta info. Así los nodos pueden publicar información a través de este pipe o leer info.

Mensaje:Cada tópico usa un tipo de msj para publicar info. Ejemp. el tópico counter usa Int 32.
Tópico trnasporta info usando mensajes.

In [ ]:
rosmsg show std_msgs/Int32 #Nos dice que atributos tiene ese msg
--> std_msgs/Int32 data #Data es su atributo

Código para crear un publisher(.py dentro de /movement/scr)

In [ ]:
#! /usr/bin/env python

import rospy 
from std_msgs.msg import Int32

rospy.init_node('topic_publisher') #Crea el publicador
pub = rospy.Publisher('/counter', Int32, queue_size = 1) #Le dice hey publicare en counter que ya se tipo msh es Int32
rate = rospy.Rate(2)
count = Int32() #Creo variable de ese tipo
count.data = 0 #Accedo al atributo del msg

while not rospy.is_shutdown():
    pub.publish(count) #publico
    count.data += 1
    rate.sleep()

In [ ]:
rosrun movement name_codigo.py
rostopic echo /counter 

In [ ]:
rospotic info /cmd_vel
--> Type geometry_msgs/Twist
rosmsg show geometry_msgs/Twist
--> float64 x ....

Se movera en circulos

In [ ]:
#! /usr/bin/env python

import rospy 
from geometry_msgs.msg import Twist

rospy.init_node('twist_publisher') #Crea el publicador
pub = rospy.Publisher('/cmd_vel', Twist, queue_size = 1) #Le dice hey publicare en counter que ya se tipo msh es Int32
rate = rospy.Rate(2)
mov = Twist() #Creo variable de ese tipo
mov.linear.x = 1 #Accedo al atributo del msg
mov.angular.z = 1

while not rospy.is_shutdown():
    pub.publish(mov) 
    rate.sleep()
    


In [ ]:
rosrun movement name_codigo.py


Si quieres que pare, pon los valores x,z = 0, sino lee el último msg publicado.

Tópicos : Subscriber

Recordemos, publisher = nodo que publica msg en un tópico
subscriber = nodo que lee msg de un Tópico
Tópico = pipe que transporta la info a través de msg.

In [ ]:
#! /usr/bin/env python
import rospy
from std_msgs.msg import Int32

def callback(msg): #Se define la funcion callbal que recibe un parametro msg
    print(msg.data) 

rospy.init_node('topic_subscriber') #Crea un objeto Susbcriber que escuchará al tópico /counter
sub = rospy.Subscriber('counter',Int32,callback) #Llamará a la función callback cada vez que lea algo del tópico
rospy.spin() #Crea un loop que mantiene el programa enejecución

Entonces, cada vez que se publique algo en counter, este subscriber lo leerá.

In [ ]:
rosrun movement name.py
#Veremos que el resultados es el mismo
#que poner rostopic echo /counter :)

In [ ]:
rosnode list 
rosnode kill /topic_publisher #mata nodos q publica

Matado el publisher que creamo, podemos crear otro ya no con código sino 
de forma manual.

In [ ]:
rostopic pub /counter std_msgs/Int32 "data:5"

Que hace lo mismo que hacía nuestro pub.py

Tarea : Robot lee del scan/laser para evitar muro y publica en cmd/vel 
Creamos un nuevo subscriptor a /kobuki/laser/scan 

In [ ]:
Averiguamos todo sobre /kobuki/laser/scan 
rostopic info /kobuki/laser/scan 
--> Type: sensor_msgs/LaserScan
rosmsg show sensor_msgs/LaserScan
--> std_msgs/Header header
    uint32 seq
    time stamp
    string frame_id
    float32 angle_min
    float32 angle_max
    float32 angle_increment
    float32 time_increment
    float32 scan_time
    float32 range_min
    float32 range_max
    float32[] ranges
    float32[] intensities

In [ ]:
#! /usr/bin/env python
import rospy
from sensor_msgs.msg import LaserScan

def callback(msg):
    #tam del vector range
    #print len(msg.ranges)
    # values at 0 degree
    print msg.ranges[0]
    # values at 90 degree
    print msg.ranges[360]
    # values at 180 degree
    print msg.ranges[719]

rospy.init_node('scan_values') ##Cuidado!pon ese mismo nombre
sub = rospy.Subscriber('/kobuki/laser/scan', LaserScan, callback)
rospy.spin()

Ahora combinamos Publish and Susbcritor

In [ ]:
#! /usr/bin/env python

import rospy 
from geometry_msgs.msg import Twist

from sensor_msgs.msg import LaserScan


def callback(msg):
    print msg.ranges[360]
    print("-----------------------")
    if msg.ranges[360] > 2:
        mov.linear.x = 0.5
        pub.publish(mov) 
    else:
        mov.linear.x = 0
        pub.publish(mov) 
        
    
rospy.init_node('twist_publisher') #Crea el publicador
pub = rospy.Publisher('/cmd_vel', Twist, queue_size = 1) #Le dice hey publicare en counter que ya se tipo msh es Int32
rate = rospy.Rate(2)
mov = Twist() #Creo variable de ese tipo
mov.linear.x = 0.5 #Accedo al atributo del msg
mov.angular.z = 0


#rospy.init_node('scan_values') 
while not rospy.is_shutdown():
    sub = rospy.Subscriber('/kobuki/laser/scan', LaserScan, callback)
    rate.sleep()
    


Se tuvo que comentar esta línea porque el nodo ya existía
rospy.init_node('scan_values') 